In [ ]:
# !pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-on125two
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-on125two
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using c

In [1]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 20.0 MB/s eta 0:00:00


In [2]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 MB 3.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-nvrtc-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-runtime-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cudnn-cu12==8.9.2.26; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requ

In [3]:
import os
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler, PolynomialFeatures,LabelEncoder
from sklearn.model_selection import train_test_split
# import clip
import torch
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, r2_score
from catboost import CatBoostRegressor, Pool
import xgboost as xgb

In [4]:
from sklearn.feature_selection import SelectFromModel

In [5]:
import cv2

In [6]:
from torchvision import transforms

In [7]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

In [8]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

if device == torch.device("cuda:0"):
  print('Everything looks good; continue')
else:
  # It is OK if you cannot connect to a GPU. In this case, training the model for
  # 2 epoch is sufficient to get full mark. (NOTE THAT 2 epoch takes approximately 1.5 hours to train for CPU)
  print('GPU is not detected. Make sure you have chosen the right runtime type')

Everything looks good; continue


In [9]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kspate","key":"f980f4b0b01e42cd75d9c93fde87386f"}'}

In [10]:
!pip install -q kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c cs-480-2024-spring
! unzip cs-480-2024-spring.zip

Streaming output truncated to the last 5000 lines.
  inflating: data/train_images/196207162.jpeg  
  inflating: data/train_images/196207164.jpeg  
  inflating: data/train_images/196207198.jpeg  
  inflating: data/train_images/196207403.jpeg  
  inflating: data/train_images/196207539.jpeg  
  inflating: data/train_images/196207572.jpeg  
  inflating: data/train_images/196207729.jpeg  
  inflating: data/train_images/196207888.jpeg  
  inflating: data/train_images/196208355.jpeg  
  inflating: data/train_images/196208682.jpeg  
  inflating: data/train_images/196208848.jpeg  
  inflating: data/train_images/196209074.jpeg  
  inflating: data/train_images/196209078.jpeg  
  inflating: data/train_images/196209236.jpeg  
  inflating: data/train_images/196209887.jpeg  
  inflating: data/train_images/196209889.jpeg  
  inflating: data/train_images/196209960.jpeg  
  inflating: data/train_images/196210323.jpeg  
  inflating: data/train_images/196211220.jpeg  
  inflating: data/train_images/196211

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


!pip install -q kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c cs-480-2024-spring
! unzip cs-480-2024-spring.zip

In [12]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
train['imagepath'] = train['id'].apply(lambda x: os.path.join('data/train_images', f"{x}.jpeg"))
test['imagepath'] = test['id'].apply(lambda x: os.path.join('data/test_images', f"{x}.jpeg"))

In [13]:
features = test.columns.values[1:-2]
targets = ['X4_mean', 'X11_mean', 'X18_mean', 'X50_mean', 'X26_mean', 'X3112_mean']

In [14]:
def remove_outliers(df, target_columns):
    for col in target_columns:
        mean = df[col].mean()
        std = df[col].std()
        df = df[(df[col] >= mean - 3 * std) & (df[col] <= mean + 3 * std)]
    return df

train = remove_outliers(train, targets)

In [15]:
train, val = train_test_split(train, test_size=0.2, random_state=42)

In [16]:
scaler = StandardScaler()
train_scale = scaler.fit_transform(train[features].values.astype(np.float32))
val_scale = scaler.transform(val[features].values.astype(np.float32))
test_scale = scaler.transform(test[features].values.astype(np.float32))

In [17]:
model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitg14_reg')
model.eval().to('cuda')

Downloading: "https://github.com/facebookresearch/dinov2/zipball/main" to /root/.cache/torch/hub/main.zip
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vitg14/dinov2_vitg14_reg4_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dinov2_vitg14_reg4_pretrain.pth
100%|██████████| 4.23G/4.23G [04:19<00:00, 17.5MB/s]


DinoVisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 1536, kernel_size=(14, 14), stride=(14, 14))
    (norm): Identity()
  )
  (blocks): ModuleList(
    (0-39): 40 x NestedTensorBlock(
      (norm1): LayerNorm((1536,), eps=1e-06, elementwise_affine=True)
      (attn): MemEffAttention(
        (qkv): Linear(in_features=1536, out_features=4608, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=1536, out_features=1536, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (drop_path1): Identity()
      (norm2): LayerNorm((1536,), eps=1e-06, elementwise_affine=True)
      (mlp): SwiGLUFFNFused(
        (w12): Linear(in_features=1536, out_features=8192, bias=True)
        (w3): Linear(in_features=4096, out_features=1536, bias=True)
      )
      (ls2): LayerScale()
      (drop_path2): Identity()
    )
  )
  (norm): LayerNorm((1536,), eps=1e-06, elementwise_affine=True)
  (head

In [18]:
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [19]:
def extract_image_embeddings(image_path):
    image = preprocess(Image.open(image_path)).unsqueeze(0).to('cuda')
    with torch.no_grad():
        image_features = model(image)
    return image_features.cpu().numpy().flatten()


In [20]:
train['image_embeddings'] = train['imagepath'].apply(extract_image_embeddings)
val['image_embeddings'] = val['imagepath'].apply(extract_image_embeddings)
test['image_embeddings'] = test['imagepath'].apply(extract_image_embeddings)

In [21]:
train_2=train
val2=val
test2=test

In [22]:
train_2.to_parquet('train_21.parquet')
val2.to_parquet('val21.parquet')
test2.to_parquet('test21.parquet')

In [23]:
!ls /content/drive/MyDrive/
!cp train_21.parquet /content/drive/MyDrive/'Colab Notebooks'/
!cp val21.parquet /content/drive/MyDrive/'Colab Notebooks'/
!cp test21.parquet /content/drive/MyDrive/'Colab Notebooks'/

'Colab Notebooks'


In [24]:
train_2 = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/train_21.parquet')
test_2 = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/test21.parquet')
val_2 = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/val21.parquet')

In [25]:
train=train_2
val=val_2
test=test_2

In [26]:
train_embeddings = pd.DataFrame(train['image_embeddings'].tolist(), index=train.index)
val_embeddings = pd.DataFrame(val['image_embeddings'].tolist(), index=val.index)
test_embeddings = pd.DataFrame(test['image_embeddings'].tolist(), index=test.index)


train = train.drop(columns=['imagepath'])
val = val.drop(columns=['imagepath'])
test = test.drop(columns=['imagepath'])


train = pd.concat([train, train_embeddings], axis=1)
val = pd.concat([val, val_embeddings], axis=1)
test = pd.concat([test, test_embeddings], axis=1)

targets = ['X4_mean', 'X11_mean', 'X18_mean', 'X50_mean', 'X26_mean', 'X3112_mean']
features = train.columns.drop(targets).drop('image_embeddings')

In [27]:
X_train_xgboost = train[features]
y_train_xgboost = train[targets]

In [28]:
X_train_xgboost

,id,WORLDCLIM_BIO1_annual_mean_temperature,WORLDCLIM_BIO12_annual_precipitation,WORLDCLIM_BIO13.BIO14_delta_precipitation_of_wettest_and_dryest_month,WORLDCLIM_BIO15_precipitation_seasonality,WORLDCLIM_BIO4_temperature_seasonality,WORLDCLIM_BIO7_temperature_annual_range,SOIL_bdod_0.5cm_mean_0.01_deg,SOIL_bdod_100.200cm_mean_0.01_deg,SOIL_bdod_15.30cm_mean_0.01_deg,...,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535
13728,159119058,17.140923,1677.750000,254.821426,58.408424,848.466064,31.346428,122,146,139,...,-0.198561,-0.293725,1.158086,1.102206,-2.735550,0.109377,0.042764,1.640329,-0.967141,0.550604
1917,195713593,14.639564,1513.500000,106.228569,23.848022,367.680389,17.873333,99,126,108,...,1.476611,0.012854,0.749141,1.955154,0.926755,0.770986,1.226421,2.608817,-0.865424,-0.101135
33680,195791141,8.018829,3003.514404,117.561905,14.784245,368.121246,18.550476,74,104,84,...,0.025668,-1.290269,0.468209,0.933641,-0.954441,0.239333,-1.086626,-2.071190,-0.756257,0.003266
26814,195286164,1.312564,412.367340,24.943878,23.431581,828.240906,39.564796,115,161,133,...,0.105486,-0.425059,-0.897845,1.370672,-0.102074,-0.913481,0.747078,0.185578,0.852758,-0.262991
4053,193699720,9.656706,1323.814331,97.352379,30.923349,681.714966,25.935715,108,144,123,...,0.622227,0.215349,-0.782557,0.923917,-1.757493,-0.650420,1.544808,2.569951,1.038464,2.347569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6994,191594720,1.008948,1167.238037,72.623810,25.296358,774.028931,28.577145,104,148,119,...,1.746698,-0.717781,-2.607017,2.708269,0.035064,-1.244583,-0.852653,2.364472,1.649897,2.287210
12580,191859470,-1.082163,718.885742,34.038094,16.439362,727.632935,32.309525,102,159,121,...,-0.123638,1.249791,0.301000,1.203452,-0.725159,0.204529,1.523153,2.220499,0.211232,1.423029
42535,195261564,2.236091,507.728577,76.085716,55.434181,1169.248413,40.813335,80,142,120,...,0.543632,0.884405,-0.248343,1.617025,0.441579,-0.104125,-1.320290,1.532026,-0.654902,-0.085830
946,108017575,5.355456,259.914276,55.123810,80.330070,461.266235,24.227142,97,129,114,...,-1.265855,2.768802,1.082967,1.591476,1.584002,1.723740,0.348080,-0.801116,1.138381,1.909159


In [29]:
train.columns = train.columns.astype(str)
val.columns = val.columns.astype(str)
test.columns = test.columns.astype(str)

In [30]:
models = {}
train_preds = {}
val_preds = {}
test_preds = {}
train_r2 = {}
val_r2 = {}

for target in targets:
    print(f"Training models for target: {target}")

    xgb_model = xgb.XGBRegressor()
    xgb_model.fit(X_train_xgboost, y_train_xgboost[target])

    importances = xgb_model.feature_importances_
    feature_importance_df = pd.DataFrame({'feature': features, 'importance': importances})
    feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)
   
    top_features = feature_importance_df['feature'][:500].astype(str).values

    pca = PCA(n_components=100, random_state=42)
    pca_train = pca.fit_transform(train[top_features])
    pca_val = pca.transform(val[top_features])
    pca_test = pca.transform(test[top_features])

    pca_columns = [f'pca_{i}' for i in range(100)]
    pca_train_df = pd.DataFrame(pca_train, columns=pca_columns, index=train.index)
    pca_val_df = pd.DataFrame(pca_val, columns=pca_columns, index=val.index)
    pca_test_df = pd.DataFrame(pca_test, columns=pca_columns, index=test.index)

    X_train = pd.concat([train[top_features], pca_train_df], axis=1)
    y_train = train[target]
    X_val = pd.concat([val[top_features], pca_val_df], axis=1)
    y_val = val[target]
    X_test = pd.concat([test[top_features], pca_test_df], axis=1)


    train_pool = Pool(X_train, y_train)
    val_pool = Pool(X_val, y_val)
    test_pool = Pool(X_test)

    model = CatBoostRegressor(
        iterations=2500,
        learning_rate=0.06,
        depth= 9,
        loss_function='RMSE',
        eval_metric='RMSE',
        random_seed=42
    )

    model.fit(
        train_pool,
        eval_set=val_pool,
        verbose=100,
        early_stopping_rounds=50
    )

    models[target] = model

    train_preds[target] = model.predict(train_pool)
    val_preds[target] = model.predict(val_pool)
    test_preds[target] = model.predict(test_pool)

    train_r2[target] = r2_score(y_train, train_preds[target])
    val_r2[target] = r2_score(y_val, val_preds[target])

    print(f"Train R² for {target}:", train_r2[target])
    print(f"Validation R² for {target}:", val_r2[target])

print("Training complete.")

Training models for target: X4_mean
0:	learn: 0.1318075	test: 0.1312575	best: 0.1312575 (0)	total: 286ms	remaining: 11m 53s
100:	learn: 0.0954173	test: 0.1024825	best: 0.1024825 (100)	total: 18.2s	remaining: 7m 11s
200:	learn: 0.0860253	test: 0.0994790	best: 0.0994790 (200)	total: 36s	remaining: 6m 51s
300:	learn: 0.0777266	test: 0.0977560	best: 0.0977560 (300)	total: 53.8s	remaining: 6m 33s
400:	learn: 0.0707737	test: 0.0967017	best: 0.0967017 (400)	total: 1m 11s	remaining: 6m 14s
500:	learn: 0.0646393	test: 0.0959289	best: 0.0959289 (500)	total: 1m 29s	remaining: 5m 56s
600:	learn: 0.0592213	test: 0.0954328	best: 0.0954320 (599)	total: 1m 47s	remaining: 5m 38s
700:	learn: 0.0543262	test: 0.0950578	best: 0.0950528 (699)	total: 2m 4s	remaining: 5m 20s
800:	learn: 0.0499888	test: 0.0946600	best: 0.0946600 (800)	total: 2m 22s	remaining: 5m 2s
900:	learn: 0.0460865	test: 0.0943919	best: 0.0943881 (899)	total: 2m 40s	remaining: 4m 44s
1000:	learn: 0.0426103	test: 0.0941273	best: 0.0941273 

In [31]:
val_r2

{'X4_mean': 0.5158511480724456,
 'X11_mean': 0.4699244750526176,
 'X18_mean': 0.614366474185604,
 'X50_mean': 0.3702960245564011,
 'X26_mean': 0.33560396543204085,
 'X3112_mean': 0.48953821023850597}

In [30]:
val_r2

{'X4_mean': 0.5128743755805528,
 'X11_mean': 0.4685291523331827,
 'X18_mean': 0.6043702245747046,
 'X50_mean': 0.37131669487402985,
 'X26_mean': 0.3268415319480965,
 'X3112_mean': 0.48031919891570973}

In [32]:
print("Validation R²:", np.mean(list(val_r2.values())))

Validation R²: 0.46593004958960255


In [33]:
submission = pd.DataFrame(test_preds)
submission = submission.rename(columns={
    'X4_mean': 'X4',
    'X11_mean': 'X11',
    'X18_mean': 'X18',
    'X50_mean': 'X50',
    'X26_mean': 'X26',
    'X3112_mean': 'X3112'
})
new_order = ['X4', 'X11', 'X18', 'X26', 'X50', 'X3112']
submission = submission[new_order]
submission.insert(0, 'id', test['id'])
submission.to_csv('submission.csv', index=False)

In [34]:
submission.head()

,id,X4,X11,X18,X26,X50,X3112
0,154220505,1.112072,143.484507,19707.344492,3504.042042,15.140369,400313.918358
1,195736552,0.976685,152.750068,19699.707261,3462.271870,15.031963,398202.680949
2,182701773,0.970125,148.708805,19698.900927,3463.774491,15.032599,398176.608368
3,27688500,0.980365,141.511378,19700.119040,3471.387316,15.981712,398192.749695
4,195825045,0.899709,152.522279,19699.029382,3464.611685,14.819483,399108.800558


In [35]:
!cp submission.csv /content/drive/MyDrive/'Colab Notebooks'/